## Motivation
_Are You the One?_ (AYTO) is an American reality TV series on MTV. The premise is simple: put 20 people in a house, and tell them if they can correctly figure out all pairs that are "perfect matches" (determined beforehand by the producers and a matchmaking algorithm), they all split a prize of $1 million.

There are two mechanisms by which the contestants can empirically narrow down their search every week:

1. Sending 1 couple to the "Truth Booth", which will reveal whether that couple is or isn't a "perfect match"
2. Sending 10 couples to the "Matching Ceremony", which will reveal the number of couples presented that are perfect matches, but not which ones were correct

The first season of this show (originally aired 2014), was just put on Netflix, and I (an avid enjoyer of trash TV) noticed that the probability of a given couple being a match could be calculated using some simple ~~statistics~~ simulations. Of course, since those episodes are over 6 years old at this point, someone (Alex Wang), already did this, and has a [blog](http://areuthe.blogspot.com/) in which he's analyzed all 8 seasons.

Admittedly, I don't have the strongest (read: any) knowledge of statistics or simulations, so I didn't completely follow how Alex was generating these probability matrixes. After some initial help from a [friend](https://github.com/prototypicalpro) in understanding the needed approach, I thought it would be fun to deconstruct exactly how to generate the probabilities of each couple as the first season of AYTO progresses.

## Framework

In season 1, the house is comprised of 10 men and 10 women, each of which are part of exactly one hetero "perfect match". Lets initialize our values, and assign names to help visualize what's going on.

In [71]:
import itertools
from enum import Enum
import math

GROUP_COUNT = 10 # Number of people in each group

class Men(Enum):
    ADAM = 0
    CHRIS_S = 1
    CHRIS_T = 2
    DILLAN = 3
    DRE = 4
    ETHAN = 5
    JOEY = 6
    JOHN = 7
    RYAN = 8
    WESLEY = 9

class Women(Enum):
    AMBER = 0
    ASHLEIGH = 1
    BRITTANY = 2
    COLEYSIA = 3
    JACY = 4
    JESSICA = 5
    KAYLA = 6
    PAIGE = 7
    SHANLEY = 8
    SIMONE = 9

print("The number of possible match permutations is: {}".format(math.factorial(GROUP_COUNT)))

The number of possible match permutations is: 3628800


There are over 3 million different ways that the set of 10 couples could be formed. Here's a random sample of a few of those:

In [72]:
import random

men_perms = list(itertools.permutations(range(GROUP_COUNT)))

def print_configurations(perms, rows):
    num_perms = len(perms)
    for i in range(rows):
        print("Configuration", i, end="\n\n")
        for w in Women:
            print("\033[96m" + w.name.center(12), end="")
        print("\033[0m")

        perm = men_perms[random.randint(0, len(perms)-1)]
        for m in perm:
            print("\033[92m" + Men(m).name.center(12), end="")
        print("\033[0m\n")
    print("... and {} more of these\n".format(num_perms-rows))
    return num_perms

num = print_configurations(men_perms, 3)

Configuration 0

   AMBER      ASHLEIGH    BRITTANY    COLEYSIA      JACY      JESSICA      KAYLA       PAIGE      SHANLEY      SIMONE   
    ADAM      CHRIS_S      WESLEY       DRE         JOEY       DILLAN      ETHAN        RYAN        JOHN      CHRIS_T   

Configuration 1

   AMBER      ASHLEIGH    BRITTANY    COLEYSIA      JACY      JESSICA      KAYLA       PAIGE      SHANLEY      SIMONE   
    JOEY        DRE         JOHN        RYAN      CHRIS_S      DILLAN       ADAM       ETHAN      CHRIS_T      WESLEY   

Configuration 2

   AMBER      ASHLEIGH    BRITTANY    COLEYSIA      JACY      JESSICA      KAYLA       PAIGE      SHANLEY      SIMONE   
   WESLEY      ETHAN        RYAN        ADAM        JOHN        DRE       CHRIS_T      DILLAN       JOEY      CHRIS_S   

... and 3628797 more of these



Notice above that the women (cyan) never change position in the list, only the men (green) are permutated. (We arbitrarily chose the men to be permuted rather than the women. If you permutated both the men and women and joined them all, you'd end up with duplicates.) This is important because it means that in our code, we can essentially only worry about permutations of men, because if a man is at a certain index, it means that he is paired with the woman associated with that index.

Now, we can start applying the conditions revealed in each episode.

## Episode 1

### Truth Booth 1

In this episode, Shanley and Chris T. are sent to the truth booth and are revealed to not be a match.
This means that we can narrow down all permutations of the couples configurations to only contain permutations that don't have Shanley and Chris T matched together.


In [73]:
# As explained above, women are represented by the index in each permutation array.
men_perms = [p for p in men_perms if p[Women.SHANLEY.value] != Men.CHRIS_T.value]

new_num = print_configurations(men_perms, 3)

print("The above permutations don't contain Shanley matched with Chris T.")
print("We've narrowed down the number of permutations from {} to {}, a reduction of {} or {:.1f}%".format(num, new_num, num-new_num, (num-new_num)/(new_num)*100))



Configuration 0

   AMBER      ASHLEIGH    BRITTANY    COLEYSIA      JACY      JESSICA      KAYLA       PAIGE      SHANLEY      SIMONE   
  CHRIS_S      ETHAN        DRE         JOEY        ADAM       WESLEY      DILLAN       JOHN        RYAN      CHRIS_T   

Configuration 1

   AMBER      ASHLEIGH    BRITTANY    COLEYSIA      JACY      JESSICA      KAYLA       PAIGE      SHANLEY      SIMONE   
   DILLAN       ADAM       ETHAN      CHRIS_T       JOHN       WESLEY       RYAN        JOEY        DRE       CHRIS_S   

Configuration 2

   AMBER      ASHLEIGH    BRITTANY    COLEYSIA      JACY      JESSICA      KAYLA       PAIGE      SHANLEY      SIMONE   
   ETHAN       WESLEY     CHRIS_S     CHRIS_T      DILLAN       RYAN        JOHN        ADAM        DRE         JOEY    

... and 3265917 more of these

The above permutations don't contain Shanley matched with Chris T.
We've narrowed down the number of permutations from 3628800 to 3265920, a reduction of 362880 or 11.1%


So how do we calculate the probability of each couple being a perfect match at this point? We simply count how many permutations are left that contain each couple, and then divide each sum by the amount of total permutations.

In [132]:

def calculate_probabilities(perms):
    pair_sum = [[0 for i in range(GROUP_COUNT)] for j in range(GROUP_COUNT)] # a zeroed two dimensional array

    # Do the counting
    for perm in perms:
        for woman, man in enumerate(perm):
            pair_sum[woman][man] += 1

    return [[pair / sum(pairs) for pair in pairs] for pairs in pair_sum]

def print_probabilities(perms):
    probabilities = calculate_probabilities(perms)
    print("".center(12), end='')
    for m in Men:
        print("\033[92m" + f'{m.name}'.center(12), end='')
    print("\033[0m")
    for woman, prob_row in enumerate(probabilities):
        print('\033[96m' + f'{Women(woman).name}'.rjust(12) + '\033[0m', end='')
        for prob_val in prob_row:
            if not prob_val:
                print("\033[91m" + 'X'.center(12) + "\033[0m", end='')
            else:
                print("\033[0m" + f'{prob_val*100:2.1f}%'.center(12) + "\033[0m", end='')
        print()
    print()

print_probabilities(men_perms)


                ADAM      CHRIS_S     CHRIS_T      DILLAN       DRE        ETHAN        JOEY        JOHN        RYAN       WESLEY   
       AMBER    8.9%        8.9%        9.8%        8.9%        8.9%        8.7%        8.9%        8.9%       19.5%        8.9%    
    ASHLEIGH    8.9%       19.5%        9.8%        8.9%        8.9%        8.7%        8.9%        8.9%        8.9%        8.9%    
    BRITTANY   19.5%        8.9%        9.8%        8.9%        8.9%        8.7%        8.9%        8.9%        8.9%        8.9%    
    COLEYSIA    8.9%        8.9%        9.8%       19.5%        8.9%        8.7%        8.9%        8.9%        8.9%        8.9%    
        JACY    8.9%        8.9%        9.8%        8.9%       19.5%        8.7%        8.9%        8.9%        8.9%        8.9%    
     JESSICA    8.7%        8.7%       22.0%        8.7%        8.7%        8.5%        8.7%        8.7%        8.7%        8.7%    
       KAYLA    8.9%        8.9%        9.8%        8.9%        8.9% 

Pretty! Shanley and Chris T. now have a slightly greater probability (relatively speaking) to be paired with anyone other than each other.

### Match Ceremony

At the end of the week, a Match Ceremony occurs. At this point, all the contestants know is that it's not Shanley and Chris T., so they pair up pretty randomly.

| Men       | Women    |
|-----------|----------|
| Wesley    | Kayla    |
| Ethan     | Shanley  |
| Adam      | Brittany |
| Dre       | Jacy     |
| John      | Simone   |
| Christ T. | Jessica  |
| Joey      | Paige    |
| Chris S.  | Ashleigh |
| Ryan      | Amber    |
| Dillan    | Coleysia |

The Match Ceremony reveals that two of these couples are correct. So how does that affect the probability of our couples?

Now, to narrow down our list of permutations, we want to only select permutations where there are exactly 2 couples in it that are the same as in the Match Ceremony.

In [134]:
# Determines if a given permutation has exactly match_count couples in it
# from the matching ceremony.
def valid_perm(perm, matches, match_count):   
    for permMan, matchMan in zip(perm, matches):
        if permMan == matchMan:
            match_count -= 1
            if match_count < 0:
                return False
    return match_count == 0

# The women are the indexes:         Amber     Ashleigh    Brittany  Colleysia   Jacy     Jessica       Kayla      Paige     Shanley    Simone
matches = tuple([x.value for x in (Men.RYAN, Men.CHRIS_S, Men.ADAM, Men.DILLAN, Men.DRE, Men.CHRIS_T, Men.WESLEY, Men.JOEY, Men.ETHAN, Men.JOHN)])
correct_match_count = 2

men_perms = [p for p in men_perms if valid_perm(p, matches, correct_match_count)]

print_probabilities(men_perms)
print("We've narrowed down the number of permutations from {} to {}, a reduction of {} or {:.1f}%".format(new_num, len(men_perms), new_num-len(men_perms), (new_num-len(men_perms))/(len(men_perms))*100))


                ADAM      CHRIS_S     CHRIS_T      DILLAN       DRE        ETHAN        JOEY        JOHN        RYAN       WESLEY   
       AMBER    8.9%        8.9%        9.8%        8.9%        8.9%        8.7%        8.9%        8.9%       19.5%        8.9%    
    ASHLEIGH    8.9%       19.5%        9.8%        8.9%        8.9%        8.7%        8.9%        8.9%        8.9%        8.9%    
    BRITTANY   19.5%        8.9%        9.8%        8.9%        8.9%        8.7%        8.9%        8.9%        8.9%        8.9%    
    COLEYSIA    8.9%        8.9%        9.8%       19.5%        8.9%        8.7%        8.9%        8.9%        8.9%        8.9%    
        JACY    8.9%        8.9%        9.8%        8.9%       19.5%        8.7%        8.9%        8.9%        8.9%        8.9%    
     JESSICA    8.7%        8.7%       22.0%        8.7%        8.7%        8.5%        8.7%        8.7%        8.7%        8.7%    
       KAYLA    8.9%        8.9%        9.8%        8.9%        8.9% 

From this, we can see the probabilities for a couple of different couples have increased, with Chris T. and Jessica now having a probability of 22.0%.